In [164]:
import urllib
from lxml import html
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
import itertools
import StringIO
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import os
%matplotlib inline

In [165]:
df = pd.read_csv('debate.csv')
df.head(n=5)

,Line,Speaker,Text,Date
0,1,Holt,Good evening from Hofstra University in Hempst...,9/26/16
1,2,Audience,(APPLAUSE),9/26/16
2,3,Clinton,"How are you, Donald?",9/26/16
3,4,Audience,(APPLAUSE),9/26/16
4,5,Holt,Good luck to you.,9/26/16


In [166]:
page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119181').read()
soup = BeautifulSoup(page, "lxml")
#speech = soup.find_all('span')[6]
speech = soup.find_all('p')
for sentence in speech:
    sentence = str(sentence)[3:-4]


In [167]:
page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119161').read()
soup = BeautifulSoup(page, "lxml")
#speech = soup.find_all('span')[6]
soup.find_all('p')

[<p>Before I begin with my remarks, I do want to say how proud I am of our brave first responders working to keep us safe after the attacks of the last weekend in New York, New Jersey and Minnesota. There are now reports of a suspect in custody, but we must remain vigilant. This is a fast-moving situation and a sobering reminder that we need steady leadership in a dangerous world.</p>,
 <p>I'm here to talk about a number of the issues that are part of this election but really, much more than that, they are part of our future \u2013 the kind of country we want to have, the kind of people we want to be and, particularly, what kind of opportunities we should be providing for the young people of America. I have a proud Owl on my staff, Jamira Burley. A Philadelphia native who became an activist working to end the epidemic of gun violence right here in Philadelphia. She loves Temple and we love her. And I also want to thank Lauren for that introduction.</p>,
 <p>Jamira and Lauren are two ex

In [168]:
valid_ids = range(165, 199)
for i in range(200,210):
    valid_ids.append(i)
trump_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #print len(speech)
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for result in speech:
        result = str(result)[3:-4]
        for i, c in enumerate(result):
            if ord(c) < 0 or ord(c) > 127:
                result = result[:i] + '.' + result[i+1:]
        sentences = tokenizer.tokenize(result)
        for sent in sentences:
            trump_speeches.append(sent)

In [169]:
print len(trump_speeches)

5739


In [170]:
def removeInvalidChars(sentences):
    lst = []
    for i, sentence in enumerate(sentences):
        segs = sentence.split('<i>')
        lst.append(''.join([segs[i] for i in range(len(segs)) if i % 2 == 0]))
    return lst

In [172]:
trump_speeches = removeInvalidChars(trump_speeches)

In [185]:
valid_ids = range(148, 165)
for i in range(497,503):
    valid_ids.append(i)
for i in range(688,702):
    valid_ids.append(i)
clinton_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for result in speech:
        result = str(result)[3:-4]
        for i, c in enumerate(result):
            if ord(c) < 0 or ord(c) > 127:
                result = result[:i] + '.' + result[i+1:]
                #print result
        sentences = tokenizer.tokenize(result)
        for sent in sentences:
            clinton_speeches.append(sent)

In [186]:
clinton_speeches = removeInvalidChars(clinton_speeches)

In [175]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#fp = open("test.txt")
#data = fp.read()



In [187]:
trump_df = pd.DataFrame({}, columns = df.columns)
clinton_df = pd.DataFrame({}, columns = df.columns)
trump = []
clinton = []

end_t = 0
end_c = 0
for i in range(df.shape[0]):
#for i in range(5):
    if df['Speaker'].values[i] == 'Trump':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            trump_df.loc[end_t] = [end_t, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            trump.append(sentence)
            end_t += 1
    if df['Speaker'].values[i] == 'Clinton':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            clinton_df.loc[end_c] = [end_c, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            clinton.append(sentence)
            end_c += 1
            
        
        
    

In [177]:
print clinton_speeches[:20]

['I want to recognize the Institute for all you do to inspire the next generation of Latino leaders.', 'Last year, I had the chance to spend time with some of the CHCI interns and fellows.', 'It really was like seeing the future of America in one room.', "I can't wait to see everything that they achieve.", "And I can't wait, if I'm fortunate enough to be president, to put some of them to work! ", 'And I want to thank all my friends in the Congressional Hispanic Caucus.', 'You fight every single day to lift up the Latino community ... when the cameras are rolling and when the cameras are off, at home in your districts and here in Washington.', "And no one understands better than you the pivotal moment we're at right now ... not just for Latinos but for our country.", 'My friends in the caucus have traveled to every battleground state, registered voters.', "You've stayed focused no matter what kind of outlandish and offensive comments we have heard from my opponent and his supporters.", 

In [178]:
print len(trump)

2020


In [179]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
for i in trump_speeches:
    trump.append(i)


In [180]:
print len(trump)

7759


In [188]:
for i in clinton_speeches:
    clinton.append(i)

In [189]:
print len(clinton)

8406


In [120]:
name = 'astro_sentences.txt'
f = open(name,'w')
'''
for sentence in trump:
    f.write(sentence+',0\n')
for sentence in clinton:
	f.write(sentence + ',1\n')
'''
tot = 0
t = 0
c = 0
cont = True
while cont:
    if np.random.uniform(low=0.0, high=1.0) < .5:
        f.write(trump[t] + ',0\n')
        tot += 1
        t += 1
    else:
        f.write(clinton[c] + ',1\n')
        tot += 1
        c += 1
    if t == len(trump):
        while c < len(clinton):
            f.write(clinton[c] + ',1\n')
            tot += 1
            c += 1
        cont = False
    elif c == len(clinton):
        while t < len(trump):
            f.write(trump[t] + ',0\n')
            tot += 1
            t += 1
        cont = False

f.close()
print tot

16165


In [70]:
print len(trump)

7759


In [118]:
name = 'trump_sentences.txt'
f = open(name,'w')
for sentence in trump:
	f.write(sentence + ' ')
f.close()

In [191]:
name = 'trump_newlines.txt'
f = open(name,'w')
for sentence in trump:
	f.write(sentence + '\n')
f.close()

In [190]:
name = 'clinton_sentences.txt'
f = open(name,'w')
for sentence in clinton:
	f.write(sentence + ' ')
f.close()

In [192]:
name = 'clinton_newlines.txt'
f = open(name,'w')
for sentence in clinton:
	f.write(sentence + '\n')
f.close()

In [32]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [39]:
for sent in trump[:3000]:
    for char in sent:
        if ord(char) < 0 or ord(char) > 127:
            print sent

SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START Let's see what happens� SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END
SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START But I will tell you, I sat there� SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END
SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START If they did it by surprise� SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END SENTENCE_END
SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START SENTENCE_START I'm here to talk about a number of the issues that are part of this election but really, much more than that, they are part of our future – the kind of country we want to have, the kind of people we want to be and, particularly, what kind of opportunities we should be providing for the young people of America. I have a

In [72]:

trump = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in trump]

trump_words = [nltk.word_tokenize(sent) for sent in trump]
#clinton_words = [nltk.word_tokenize(sent) for sent in clinton]
print type(trump)

<type 'list'>


In [73]:
word_freq = nltk.FreqDist(itertools.chain(*trump_words))

In [74]:
print type(word_freq)
print len(word_freq)
word_freq

<class 'nltk.probability.FreqDist'>
2243


FreqDist({'1,800': 3,
          'four': 5753,
          'Does': 1,
          '2,200': 2,
          'saved': 1,
          'relationships': 5,
          'increase': 5,
          'votes': 1,
          'voter': 1,
          'tweet': 1,
          'Ronald': 3,
          'voted': 2,
          'whatsoever': 1,
          'under': 12,
          'imperative': 1,
          'sorry': 1,
          'worth': 4,
          'every': 10,
          'Old': 4,
          'bringing': 8,
          'anemic': 1,
          'vast': 2,
          'companies': 33,
          'Go': 3,
          'deplorable': 1,
          'expanded': 1,
          'force': 2,
          'leaders': 9,
          'tired': 2,
          'elections': 1,
          'surrounding': 1,
          'second': 3,
          'street': 3,
          'even': 27,
          'Arizona': 1,
          'hate': 4,
          'This': 10,
          'Wrong': 11,
          'above': 1,
          'new': 14,
          'ever': 31,
          'told': 4,
          'Made': 1,
     

In [75]:
vocab_size = 2200
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Find most frequently used words
vocab = word_freq.most_common(vocab_size-1)

# Get an index_to_word vector and a word_to_index vector
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [76]:
# Substitute unknown token for words not in vocab
for i, sent in enumerate(trump_words):
    trump_words[i] = [w if w in word_to_index else unknown_token for w in sent]

In [77]:
# Training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in trump_words])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in trump_words])

In [78]:
X_train[:10]

array([[3, 806, 0, 5, 291, 4], [3, 226, 128, 40, 1633, 22, 51, 4],
       [3, 71, 35, 38, 24, 376, 4], [3, 71, 35, 38, 24, 80, 99, 173, 4],
       [3, 63, 65, 47, 11, 244, 34, 94, 24, 44, 51, 30, 394, 6, 504, 44, 833, 4],
       [3, 71, 35, 1624, 120, 1392, 5, 7, 114, 9, 285, 30, 44, 537, 24, 313, 69, 4],
       [3, 28, 29, 27, 25, 41, 137, 313, 4],
       [3, 28, 29, 27, 25, 948, 313, 4],
       [3, 207, 33, 35, 2199, 44, 51, 81, 25, 1475, 1413, 24, 1690, 244, 5, 7, 80, 99, 173, 40, 94, 22, 317, 112, 4],
       [3, 87, 29, 35, 485, 44, 137, 128, 5, 56, 80, 6, 69, 4]], dtype=object)

In [79]:
y_train[:10]

array([[806, 0, 5, 291, 4, 2], [226, 128, 40, 1633, 22, 51, 4, 2],
       [71, 35, 38, 24, 376, 4, 2], [71, 35, 38, 24, 80, 99, 173, 4, 2],
       [63, 65, 47, 11, 244, 34, 94, 24, 44, 51, 30, 394, 6, 504, 44, 833, 4, 2],
       [71, 35, 1624, 120, 1392, 5, 7, 114, 9, 285, 30, 44, 537, 24, 313, 69, 4, 2],
       [28, 29, 27, 25, 41, 137, 313, 4, 2],
       [28, 29, 27, 25, 948, 313, 4, 2],
       [207, 33, 35, 2199, 44, 51, 81, 25, 1475, 1413, 24, 1690, 244, 5, 7, 80, 99, 173, 40, 94, 22, 317, 112, 4, 2],
       [87, 29, 35, 485, 44, 137, 128, 5, 56, 80, 6, 69, 4, 2]], dtype=object)

In [80]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [81]:
class RNNNumpy:
     
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))


In [82]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]
 
RNNNumpy.forward_propagation = forward_propagation

In [83]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)
 
RNNNumpy.predict = predict

In [84]:
np.random.seed(10)
model = RNNNumpy(vocab_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(28, 2200)
[[ 0.00046266  0.0004612   0.00045563 ...,  0.00045901  0.00045776
   0.00045215]
 [ 0.00045574  0.00045031  0.00045436 ...,  0.00045571  0.00045103
   0.00045817]
 [ 0.00045421  0.00044945  0.0004534  ...,  0.00046162  0.00045075
   0.00046014]
 ..., 
 [ 0.00045515  0.00044997  0.00045885 ...,  0.00044881  0.00046394
   0.00045005]
 [ 0.00046002  0.00045328  0.00045665 ...,  0.00046075  0.00045585
   0.00045195]
 [ 0.00045103  0.00045663  0.00044964 ...,  0.00046408  0.00045574
   0.00045365]]


In [85]:
predictions = model.predict(X_train[10])
print predictions.shape
print predictions

(28,)
[ 953 1887  981 2081    8  910 2182  294  285 1018 1347 2110 1670  881 1435
 1307 1682 1039 1416   54  290 1943   15  998 1564 1943 2164 1760]


In [86]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L
 
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N
 
RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [87]:
# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocab_size)
print "Actual loss: %f" % model.calculate_loss(X_train, y_train)

Expected Loss for random predictions: 7.696213
Actual loss: 7.698439


In [88]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt

In [89]:
import operator
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
 
RNNNumpy.gradient_check = gradient_check
 
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])


Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [90]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNNNumpy.sgd_step = numpy_sdg_step


In [91]:
import datetime
import sys
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [92]:
np.random.seed(10)
model = RNNNumpy(vocab_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

1 loop, best of 3: 64.4 ms per loop


In [162]:


np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocab_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=100, evaluate_loss_after=1)


2016-12-11 18:10:23: Loss after num_examples_seen=0 epoch=0: 7.695702
2016-12-11 18:10:29: Loss after num_examples_seen=100 epoch=1: 7.677226
2016-12-11 18:10:34: Loss after num_examples_seen=200 epoch=2: 7.644804
2016-12-11 18:10:39: Loss after num_examples_seen=300 epoch=3: 7.559915
2016-12-11 18:10:44: Loss after num_examples_seen=400 epoch=4: 5.801030
2016-12-11 18:10:48: Loss after num_examples_seen=500 epoch=5: 5.332393
2016-12-11 18:10:53: Loss after num_examples_seen=600 epoch=6: 5.127413
2016-12-11 18:10:58: Loss after num_examples_seen=700 epoch=7: 5.004839
2016-12-11 18:11:03: Loss after num_examples_seen=800 epoch=8: 4.918421
2016-12-11 18:11:08: Loss after num_examples_seen=900 epoch=9: 4.850562
2016-12-11 18:11:15: Loss after num_examples_seen=1000 epoch=10: 4.789953
2016-12-11 18:11:20: Loss after num_examples_seen=1100 epoch=11: 4.730197
2016-12-11 18:11:25: Loss after num_examples_seen=1200 epoch=12: 4.672401
2016-12-11 18:11:29: Loss after num_examples_seen=1300 epoch

In [163]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)[0]
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            #print type(next_word_probs[-1])
            #print next_word_probs[-1].shape
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    #print str(len(index_to_word)) + ' index_to_word'
    #for x in new_sentence[1:-1]:
        #print x
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str
 
num_sentences = 10
senten_min_length = 7
 
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

New lost 's been thing paying .
Now 's you leaving looking you cars job .
They did are , agreement do government better debt , and of the world n't that frankly .
And wages should do look for in 3,000 .
The will no do we for much , taxes now cars our .
And we are to stop lot is a you at to in about , and I as 'd like cases , they or because these talking done these invest solutions .
almost , I 's to some in the right Florida our it of Michigan percent out our making but happen with my thousands since and Democratic solutions 're n't our or of , early , thinking these to doing it .
And we you to what Clinton is years to doing tax $ 20 of like Hillary companies us of want watch you they the innovative .
And talks on stop say country much agreement when do a world , to new Reagan build make that disaster ever , and they the never them jobs very that relief debt countries in done of the country and our 've of since of debt .
And we also to stop Clinton .


In [95]:
sentences_df.head(n=5)

NameError: name 'sentences_df' is not defined

In [ ]:
x_text = []
y_cand = []
for i in range(len(df['Speaker'].values)):
    if df['Speaker'].values[i] == 'Trump':
        x_text.append(df['Text'].values[i])
        y_cand.append('Trump')
    elif df['Speaker'].values[i] == 'Clinton':
        x_text.append(df['Text'].values[i])
        y_cand.append('Clinton')
        

In [3]:
c_text = []
t_text = []
for i in range(len(df['Speaker'].values)):
    if df['Speaker'].values[i] == 'Trump':
        t_text.append(df['Text'].values[i])
    elif df['Speaker'].values[i] == 'Clinton':
        c_text.append(df['Text'].values[i])
        

In [5]:
name = 'clintonwords.txt'
f = open(name,'w')
for i in range(len(c_text)):
	f.write(c_text[i] + ' ')
f.close()

In [6]:
name = 'trumpwords.txt'
f = open(name,'w')
for i in range(len(t_text)):
	f.write(t_text[i] + ' ')
f.close()

In [34]:
vectorizer = CountVectorizer(encoding='latin1', stop_words=['and', 'or', 'before', 'a', 'an', 'am', 'the', 'at', 'by', 'br'], min_df=4)
x = vectorizer.fit_transform(x_text)
x = x.toarray()
print type(x)

<type 'numpy.ndarray'>


In [26]:
log = LogReg()
log.fit(x, y_cand)
log.score(x, y_cand)

0.95986622073578598

In [37]:
print len(x)
print len(x[0])
print len(vectorizer.vocabulary_)
print len(x_text)

598
982
982
598


In [41]:
word_counts = zip(x_text, x)

In [ ]:
#features = []
for i in range(len(x_text)):
    if 